---
## DEMO: Applying a cascade of two EZ-FRCNN models to analyze an example video of a single worm
In this demo, you will use two pre-trained EZ-FRCNN models* to analyze a video of feeding *C. elegans* on food. The steps are as follows:
1. Apply the pharyngeal bulb-tracking EZ-FRCNN model to the original 4k video.
2. Use the tracked bulb location to dynamically crop the original 4k video into a 250x250p video centered on the bulb.
3. Apply the grinder-tracking EZ-FRCNN model to the 250x250p video centered on the grinder.

>*NOTE: You will need to download the models from our OSF, located [here](https://osf.io/z7t9s/). Navigate to the Files tab, then click `c. elegans grinder tracking/models` to locate and download each model. After downloading, move the models to the `demo_wormmtg/models` folder within this repository.

>Example videos are also available on [OSF](https://osf.io/z7t9s/) to download under `c. elegans grinder tracking/videos/demo_wormmtg`

### Motivation
We use EZ-FRCNN both as a preprocessing step as well as an object detector in tracking the grinder in freely moving *C. elegans*. In a 4K video taken at 6X magnification on a 10X objective, the grinder appears as a dark, 20x20 pixel "blob" that can be difficult to track in the raw video data. To circumvent this, we first train an EZ-FRCNN model to track the pharyngeal bulb: the part of the pharynx that contains the grinder. Then, we use this bulb location to crop an ROI centered on the grinder. Using this ROI video, we can then apply a 2nd EZ-FRCNN model to track the grinder reliably.

In [1]:
# --- LOAD LIBRARIES + CHECK THAT GPU IS AVAILABLE ---
from library import *     # imports all libraries needed to run this code
torch.cuda.is_available() # check that the GPU is available, will print 'True' if it is 
                          # (not REQUIRED, but highly RECOMMENDED)

True

---
### Step 1: Apply pharyngeal bulb tracking
This step will apply a pretrained bulb tracking EZ-FRCNN model to a video. The output of this step is as follows: 
1. A CSV listing out the bounding box location of the bulb at each frame of the video, located at `demo_wormmtg/VIDEONAME_bulb_boxes.csv`
2. A labeled version of the video, located at `demo_wormmtg/VIDEONAME_labeled.avi`

In [2]:
# --- LOAD PRE-TRAINED BULB TRACKING MODEL ---
detection_threshold = 0.7 # this threshold determines how high the confidence score for an 
                          # inference must be to be considered a correct label

MODEL_DIR   = './demo_wormmtg/models'
CLASSES     = ['background','bulb']
NUM_CLASSES = 2
model_name  = 'example_bulb_model.pth'
model       = load_model(model_name, MODEL_DIR, NUM_CLASSES)

In [3]:
# --- APPLY PRE-TRAINED BULB TRACKING MODEL TO VIDEO ---
video        = 'example_video_onfood'
video_name   = video + '.wmv'
VID_DIR      = './demo_wormmtg/videos/' + video_name
OUT_VID_DIR  = './demo_wormmtg/outputs/' + video + '_labeled.avi'
BULB_BOX_DIR = './demo_wormmtg/outputs/' + video + '_bulb_boxes.csv'

# This function will apply the model to the video
[bboxes, classes, scores] = inference_video_clean(VID_DIR, model, detection_threshold, CLASSES)

# This function will save the bounding box locations
saveBestBoxes(BULB_BOX_DIR, bboxes)

# This function will generate a video with the bounding boxes labeled
save_clean_video(VID_DIR, BULB_BOX_DIR, OUT_VID_DIR, lw=3, color=(51, 114, 252))

Video saved to  ./demo_wormmtg/outputs/example_video_onfood_labeled.avi


---
### Step 2: Crop the original video using the tracked pharyngeal bulb
This step will use the tracked bulb from Step 1 to crop an ROI from each frame of the selected video. The output of this step is as follows: 
1. A cropped version of the original video, located at `demo_wormmtg/videos/VIDEONAME_cropped.avi`

In [5]:
# --- PARAMETERS ---
ROI_width   = 125 # resulting video will be 2*roi_width x 2*roi_width
thresh, sig = 250, 6 # threshold for filtering ROI coordinates, sigma used for smoothing the ROI centers
frame_size  = 2 * ROI_width, 2 * ROI_width

# --- LOAD BOUNDING BOXES & VIDEO ---
boxes = pd.read_csv(BULB_BOX_DIR).to_numpy()
vid   = cv2.VideoCapture(VID_DIR)
width, height = int(vid.get(3)), int(vid.get(4))
fps   = vid.get(cv2.CAP_PROP_FPS)
tot_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

# --- OUTPUT SETUP ---
out_path = './demo_wormmtg/videos/' + video + '_cropped.wmv'
vwriter = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'MJPG'), fps, frame_size)

# --- COMPUTE ROI CENTERS ---
ROI_coords = np.array([
    [(box[0] + box[2]) // 2, (box[1] + box[3]) // 2]
    for box in boxes[:tot_frames - 1]
])

# --- FILTER & SMOOTH COORDS ---
ROI_coords_i, ROI_coords_s = filter_ROIs(ROI_coords, threshold=thresh, sigma=sig, max_values=(3840, 2160))

# --- PROCESS VIDEO ---
for f in range(1, tot_frames-1):
    success, frame = vid.read()
    if not success: break

    print(f"frame: {f-1}")
    ROI = getROI(ROI_width, ROI_coords_s[f], frame, width, height)
    vwriter.write(ROI)
    clear_output(wait=True)

# --- CLEANUP ---
vwriter.release()
clear_output(wait=True)
print("Saved to", out_path)

Saved to ./demo_wormmtg/videos/example_video_onfood_cropped.wmv


---
### Step 3: Apply grinder tracking
This step will apply a pretrained grinder tracking EZ-FRCNN model to an ROI video generated in Step 2. The output of this step is as follows: 
1. A CSV listing out the bounding box location of the grinder at each frame of the video, located at `demo_wormmtg/VIDEONAME_grinder_boxes.csv`
2. A labeled version of the ROI video, located at `demo_wormmtg/VIDEONAME_cropped_labeled.avi`

In [4]:
### LOAD PRE-TRAINED GRINDER TRACKING MODEL
detection_threshold = 0.5 # this threshold determines how high the confidence score for an 
                          # inference must be to be considered a correct label
CLASSES    = ['background','grinder']
model_name = 'example_grinder_model.pth'
model      = load_model(model_name, MODEL_DIR, NUM_CLASSES)

In [6]:
### APPLY PRE-TRAINED GRINDER TRACKING MODEL TO CROPPED VIDEO
video           = 'example_video_onfood'
video_name      = video + '_cropped.wmv'
VID_DIR         = './demo_wormmtg/videos/' + video_name
OUT_VID_DIR     = './demo_wormmtg/outputs/' + video + '_cropped_labeled.avi'
GRINDER_BOX_DIR = './demo_wormmtg/outputs/' + video + '_grinder_boxes.csv'

# This function will apply the model to the video
[bboxes, classes, scores] = inference_video_clean(VID_DIR, model, detection_threshold, CLASSES)

# This function will save the bounding box locations
saveBestBoxes(GRINDER_BOX_DIR, bboxes)

# This function will generate a video with the bounding boxes labeled
save_clean_video(VID_DIR, GRINDER_BOX_DIR, OUT_VID_DIR, lw=2, color=(51, 114, 252))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

